In [1]:
import pandas as pd
from google.cloud import bigquery
from google.cloud.bigquery import job
from datetime import date, timedelta
# 판다스 엑셀 익스포트를 위한 라이브러리
import openpyxl


PROJECT = "ballosodeuk"
bq = bigquery.Client(project=PROJECT)


query = """
select a.*, survival_prob, is_churn, current_shoji, frequency, recency, monetary, grade, loyalty_segment, days_since_last_activity
from
  (
    select * from `dm.agg_user_category_rank`
    where power_1_2 is not null
  ) a
left join
  (
    select user_id, survival_prob, is_churn, loyalty_segment, days_since_last_activity
    from `dm.agg_user_churn_daily`
    where cur_date = current_date() - 1
  ) b on a.user_id = b.user_id
left join 
  (
    select *
    from `dm.agg_user_rfm_daily`
    where register_dt = current_date() -1
  ) c on a.user_id = c.user_id
"""

df = bq.query(query).to_dataframe()

c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


In [17]:
df.to_excel('seg_power.xlsx', engine='openpyxl', encoding='utf-8-sig', index=False)

c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [2]:
df.is_churn.value_counts()

False    4653
True     1580
Name: is_churn, dtype: Int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6268 entries, 0 to 6267
Data columns (total 45 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   user_id                   6268 non-null   object 
 1   latest_order_dt           6268 non-null   dbdate 
 2   total_purchase_count      6268 non-null   Int64  
 3   ranking_1_1               6268 non-null   object 
 4   power_1_1                 6268 non-null   float64
 5   ranking_1_2               6268 non-null   object 
 6   power_1_2                 6268 non-null   float64
 7   ranking_1_3               1242 non-null   object 
 8   power_1_3                 1242 non-null   float64
 9   ranking_1_4               339 non-null    object 
 10  power_1_4                 339 non-null    float64
 11  ranking_1_5               103 non-null    object 
 12  power_1_5                 103 non-null    float64
 13  ranking_2_1               6268 non-null   object 
 14  power_2_

In [14]:
import re

power_cols = [col for col in df.columns.tolist() if col.startswith('power')]
ranking_cols = [col for col in df.columns.tolist() if col.startswith('ranking')]
# df[power_cols] = df[power_cols].fillna(0)
# df[power_cols] = df[power_cols].astype(float)

# df['power_1_2'] = df['power_1_2'].fillna(0)
# df['power_1_2'] = df['power_1_2'].astype(float)




In [22]:
ranking_cols

['ranking_1_1',
 'ranking_1_2',
 'ranking_1_3',
 'ranking_1_4',
 'ranking_1_5',
 'ranking_2_1',
 'ranking_2_2',
 'ranking_2_3',
 'ranking_2_4',
 'ranking_2_5',
 'ranking_2_6',
 'ranking_2_7',
 'ranking_2_8',
 'ranking_2_9',
 'ranking_2_10']

In [26]:
df

,user_id,latest_order_dt,total_purchase_count,ranking_1_1,power_1_1,ranking_1_2,power_1_2,ranking_1_3,power_1_3,ranking_1_4,...,depth3_list,survival_prob,is_churn,current_shoji,frequency,recency,monetary,grade,loyalty_segment,days_since_last_activity
0,5aeb3860-5cb8-4275-95e1-9a0942999b3a,2025-03-26,2,패션의류,0.622032,식품,0.377968,None,0.000000,None,...,"식품(음료[생수]), 패션의류(남성언더웨어-잠옷[팬티])",0.786821,False,0,2,4,49900.0,VIP,장기 충성 유저,4
1,c0618b2c-1d53-439f-a6b1-b2842ce989a5,2025-03-26,2,패션잡화,0.500000,식품,0.500000,None,0.000000,None,...,"패션잡화(양말[여성양말]), 식품(가공식품[통조림])",0.786821,False,178,1,4,26700.0,GOLD,장기 충성 유저,4
2,ec8fd318-4a79-45fc-ab08-4d98612e9592,2025-03-26,2,식품,0.643930,패션잡화,0.356070,None,0.000000,None,...,"패션잡화(모자[야구모자]), 식품(음료[생수])",1.000000,False,15514,2,4,24200.0,VIP,장기 충성 유저,0
3,66e99721-0860-4784-8ce9-6174bfa1cfd4,2025-03-26,2,식품,0.632883,생활-건강,0.367117,None,0.000000,None,...,"식품(과자[쿠키]), 생활-건강(주방용품[주방잡화])",0.741831,False,0,2,4,25700.0,VIP,충성 유저,4
4,dd9fd167-f798-4b96-8e5a-3a113ac584f0,2025-03-26,2,식품,0.632883,생활-건강,0.367117,None,0.000000,None,...,"식품(수산[젓갈]), 생활-건강(구강위생용품[칫솔])",0.769185,False,0,2,4,20700.0,VIP,충성 유저,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6263,c26404f8-5f38-43f6-b052-9df694c3b8e1,2025-03-28,2,식품,0.619595,화장품-미용,0.380405,None,0.000000,None,...,"식품(음료[생수]), 화장품-미용(바디케어[핸드케어])",1.000000,False,1168,2,2,12900.0,GOLD,장기 충성 유저,0
6264,222bf5d1-ad4d-41f7-9e0b-59f5ca64eb7e,2024-11-05,6,화장품-미용,0.500000,패션잡화,0.166667,식품,0.166667,패션의류,...,"패션잡화(양말[남성양말]), 화장품-미용(스킨케어[화장품세트]), 화장품-미용(바디...",1.000000,False,66242,2,145,132500.0,GOLD,장기 충성 유저,0
6265,7602688c-f442-4873-b32b-0ffd1c65d349,2024-11-05,3,식품,0.666667,생활-건강,0.333333,None,0.000000,None,...,"식품(냉동-간편조리식품[즉석국]), 생활-건강(주방용품[조리기구]), 식품(음료[생수])",0.057031,False,78,1,145,50200.0,IRON,정착기 유저,20
6266,f75bdcf7-f6bf-41c7-be67-cc253d373bf7,2024-11-05,2,가구-인테리어,0.500000,생활-건강,0.500000,None,0.000000,None,...,"가구-인테리어(침구단품[차렵이불]), 생활-건강(주방용품[주방잡화])",0.022737,False,42867,2,145,62900.0,GOLD,안정기 유저,26


In [21]:
tst = df[ranking_cols+power_cols].copy()
power =  df[power_cols]
power

,power_1_1,power_1_2,power_1_3,power_1_4,power_1_5,power_2_1,power_2_2,power_2_3,power_2_4,power_2_5,power_2_6,power_2_7,power_2_8,power_2_9,power_2_10
0,0.622032,0.377968,0.000000,0.000000,0.0,0.622032,0.377968,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.500000,0.500000,0.000000,0.000000,0.0,0.500000,0.500000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.643930,0.356070,0.000000,0.000000,0.0,0.643930,0.356070,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.632883,0.367117,0.000000,0.000000,0.0,0.632883,0.367117,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.632883,0.367117,0.000000,0.000000,0.0,0.632883,0.367117,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6263,0.619595,0.380405,0.000000,0.000000,0.0,0.619595,0.380405,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
6264,0.500000,0.166667,0.166667,0.166667,0.0,0.333333,0.166667,0.166667,0.166667,0.166667,0.0,0.0,0.0,0.0,0.0
6265,0.666667,0.333333,0.000000,0.000000,0.0,0.333333,0.333333,0.333333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
6266,0.500000,0.500000,0.000000,0.000000,0.0,0.500000,0.500000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [25]:
tst

,ranking_1_1,ranking_1_2,ranking_1_3,ranking_1_4,ranking_1_5,ranking_2_1,ranking_2_2,ranking_2_3,ranking_2_4,ranking_2_5,...,power_2_1,power_2_2,power_2_3,power_2_4,power_2_5,power_2_6,power_2_7,power_2_8,power_2_9,power_2_10
0,패션의류,식품,None,None,None,패션의류(남성언더웨어-잠옷),식품(음료),None,None,None,...,0.622032,0.377968,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,패션잡화,식품,None,None,None,식품(가공식품),패션잡화(양말),None,None,None,...,0.500000,0.500000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,식품,패션잡화,None,None,None,식품(음료),패션잡화(모자),None,None,None,...,0.643930,0.356070,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,식품,생활-건강,None,None,None,식품(과자),생활-건강(주방용품),None,None,None,...,0.632883,0.367117,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,식품,생활-건강,None,None,None,식품(수산),생활-건강(구강위생용품),None,None,None,...,0.632883,0.367117,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6263,식품,화장품-미용,None,None,None,식품(음료),화장품-미용(바디케어),None,None,None,...,0.619595,0.380405,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
6264,화장품-미용,패션잡화,식품,패션의류,None,화장품-미용(스킨케어),식품(김치),패션잡화(양말),화장품-미용(바디케어),패션의류(남성언더웨어-잠옷),...,0.333333,0.166667,0.166667,0.166667,0.166667,0.0,0.0,0.0,0.0,0.0
6265,식품,생활-건강,None,None,None,식품(음료),생활-건강(주방용품),식품(냉동-간편조리식품),None,None,...,0.333333,0.333333,0.333333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
6266,가구-인테리어,생활-건강,None,None,None,가구-인테리어(침구단품),생활-건강(주방용품),None,None,None,...,0.500000,0.500000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [48]:
# tst[['ranking_1_1','power_1_1']].melt(id_vars=['ranking_1_1'], value_vars=['power_1_1'])
tst[['ranking_1_1','power_1_1']].melt(id_vars=['ranking_1_1'], value_vars=['power_1_1']).reset_index()\
    .pivot(index='index', columns='ranking_1_1', values='value')

ranking_1_1,가구-인테리어,디지털-가전,생활-건강,스포츠-레저,식품,출산-육아,패션의류,패션잡화,화장품-미용
index,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,0.622032,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.500000,NaN
2,NaN,NaN,NaN,NaN,0.643930,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,0.632883,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,0.632883,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6263,NaN,NaN,NaN,NaN,0.619595,NaN,NaN,NaN,NaN
6264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5
6265,NaN,NaN,NaN,NaN,0.666667,NaN,NaN,NaN,NaN


In [24]:
for i in range(1,16):
    # power[f'rank_power_cat_{i}'] = df[f'ranking_{i}_1'] * df[f'power_1_{i}']
    df[f'ranking_{i}_1'] * df[f'power_1_{i}']


TypeError: can't multiply sequence by non-int of type 'float'